1. Proportional Gain (Kp)
Effect:
    Increases the speed of response (faster rise time).
    Reduces the steady-state error.
Drawback:
    High Kp can lead to overshoot and oscillations.
    Too high may cause instability.

2. Integral Gain (Ki)
Effect:
    Eliminates steady-state error by integrating the error over time.
Drawback:
    Increases overshoot.
    Can significantly slow down settling time.
    May cause instability or oscillations if too large due to integral windup.

3. Derivative Gain (Kd)
Effect:

    Predicts system behavior and adds damping.
    Reduces overshoot and improves stability.
    Helps reduce oscillations.
Drawback:
    Too high Kd can amplify noise.
    May slow down the system if excessive.

Tuning Goal Summary
Faster rise time: Increase Kp

Reduce steady-state error: Increase Ki

Reduce overshoot & oscillations: Increase Kd

Stable and fast settling: Careful balance of all three



Here are the step response curves for different PID configurations:

P-only (Kp = 5): Fast rise time but noticeable steady-state error.

PI (Kp = 5, Ki = 2): Eliminates steady-state error, but introduces overshoot and slower settling.

PD (Kp = 5, Kd = 10): Reduces overshoot and provides a smoother response, but still some steady-state error.

Full PID (Kp = 15, Ki = 2, Kd = 10): Fast and accurate response with minimal overshoot and no steady-state error.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the plant dynamics
def plant_dynamics(x, t, u_func):
    x1, x2 = x
    u = u_func(t)
    dx1dt = x2
    dx2dt = -2 * x2 - x1 + u
    return [dx1dt, dx2dt]

# PID simulation
def simulate_pid(Kp, Ki, Kd, t):
    dt = t[1] - t[0]
    e_prev = 0
    integral = 0
    x = [0.0, 0.0]
    x_hist = []

    for i in range(len(t)):
        y = x[0]
        e = 1.0 - y  # Step input
        integral += e * dt
        derivative = (e - e_prev) / dt if i > 0 else 0
        u = Kp * e + Ki * integral + Kd * derivative
        e_prev = e
        x = odeint(plant_dynamics, x, [t[i], t[i]+dt], args=(lambda _: u,))[-1]
        x_hist.append(x[0])

    return np.array(x_hist)

# Time vector
t = np.linspace(0, 10, 1000)

# Plotting function
def plot_response(Kp, Ki, Kd):
    y = simulate_pid(Kp, Ki, Kd, t)
    plt.figure(figsize=(10, 5))
    plt.plot(t, y, label=f"Kp={Kp}, Ki={Ki}, Kd={Kd}")
    plt.title("PID Step Response")
    plt.xlabel("Time (s)")
    plt.ylabel("Output")
    plt.ylim(0, 2)
    plt.grid(True)
    plt.legend()
    plt.show()

# Create interactive widgets
kp_slider = widgets.FloatSlider(value=5.0, min=0, max=20, step=0.5, description='Kp:')
ki_slider = widgets.FloatSlider(value=1.0, min=0, max=10, step=0.1, description='Ki:')
kd_slider = widgets.FloatSlider(value=1.0, min=0, max=10, step=0.1, description='Kd:')

ui = widgets.VBox([kp_slider, ki_slider, kd_slider])
out = widgets.interactive_output(plot_response, {
    'Kp': kp_slider,
    'Ki': ki_slider,
    'Kd': kd_slider
})

display(ui, out)


Output()

Summary of Effects on Transient Parameters
Parameter	Rise Time	Overshoot	Settling Time	Steady-State Error	Stability
Kp   ↑	        ↓	        ↑	        ↔ / ↑	            ↓	            ↓ if too high
Ki   ↑	        ↓	        ↑	            ↑	            ↓	            ↓
Kd   ↑	        ↑ / ↔	    ↓	            ↓	            ↔	            ↑

Tuning Goal Summary
Faster rise time: Increase Kp

Reduce steady-state error: Increase Ki

Reduce overshoot & oscillations: Increase Kd

Stable and fast settling: Careful balance of all three